In [ ]:
import surp

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from astropy.table import Table

import arya

In [ ]:
filename = "../../data/allStar-dr17-synspec_rev1.fits"

allStar = Table.read(filename, format="fits", hdu=1, memmap=True)

In [ ]:
filt = np.full(len(allStar), True)

apogee_aspcapflag = 1<<23 # starbad

filt &= allStar["ASPCAPFLAG"] & apogee_aspcapflag == 0
filt &= allStar["EXTRATARG"] == 0
filt &= allStar["SNR"] >= 80


filt &= allStar["LOGG"] >= 1
filt &= allStar["LOGG"] <= 3.8
filt &= allStar["TEFF"] >= 3400
filt &= allStar["TEFF"] <= 5500


filt &= (allStar["LOGG"] < 3 ) | (allStar["TEFF"] > 4000)


In [ ]:
valid_id = allStar["APOGEE_ID"][filt]

In [ ]:
filename = "../../data/apogee_astroNN-DR17.fits"
astroNN = Table.read(filename, format="fits", memmap=True)

In [ ]:
np.sum(filt)

In [ ]:
filt_ann = astroNN["LOGG"] >= 0
astroNN["tau"] = astroNN["age_lowess_correct"]
filt_ann &= astroNN["tau"] <= 10
filt_ann &= astroNN["galr"] <= 15
#filt_ann &= astroNN["age_total_error"] <= 3
filt_ann &= np.abs(astroNN["galz"]) <= 0.5

In [ ]:
sum(filt_ann)

In [ ]:
filt_ann &= np.isin(astroNN["APOGEE_ID"], valid_id)

In [ ]:
sum(filt_ann)

In [ ]:
df = astroNN[filt_ann].to_pandas()

In [ ]:
plt.hist2d(astroNN["TEFF"], astroNN["LOGG"], bins=100, range=((0, 10_000), (1, 5)))
plt.gca().invert_xaxis()
plt.gca().invert_yaxis()
plt.xlabel("teff")
plt.ylabel("logg")

In [ ]:
plt.scatter(df.TEFF, df.LOGG, c=df.age, s=1)
plt.gca().invert_xaxis()
plt.gca().invert_yaxis()
plt.xlabel("teff")
plt.ylabel("logg")

In [ ]:
plt.scatter(df.galr, df.age, s=1, c=df.age_total_error, vmax=3)

In [ ]:
arya.medianplot(df, "galr", "age", binsize=1000)

In [ ]:
arya.medianplot(df, "galr", "O_H", binsize=1000, stat="median")

In [ ]:
hm = arya.HueMap((0, 10))

for low in range(10):
    high = low + 1
    filt = df.age >= low
    filt &= df.age <= high
    arya.medianplot(df[filt], "galr", "O_H", binsize=500, stat="median", errorbar=None, color=hm(low), aes="line")

In [ ]:
hm = arya.HueMap((0, 10))

for low in range(10):
    high = low + 1
    filt = df.age >= low
    filt &= df.age <= high
    arya.medianplot(df[filt], "galr", "FE_H", binsize=500, stat="median", errorbar=None, color=hm(low), aes="line")